spark实现标签传播算法的流程如下：   

0.将每个顶点ID设置为其初始标签；   
1.每个顶点将其标签发送给所有邻居；   
2.将所有邻居的标签进行统计，将出现次数最多的标签更新为当前顶点的新标签；  

重复1～2步N次

In [1]:
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.graphx._
val sparkConf: SparkConf = new SparkConf().setMaster("local[2]")
val sc: SparkContext = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@7b107f2a
sc = org.apache.spark.SparkContext@4d82496


org.apache.spark.SparkContext@4d82496

In [2]:
val initGraph=util.GraphGenerators.rmatGraph(sc,32,64)

initGraph = org.apache.spark.graphx.impl.GraphImpl@ef1b04e


org.apache.spark.graphx.impl.GraphImpl@ef1b04e

In [3]:
//将顶点ID设置为标签
val lpaGraph = initGraph.mapVertices{case (vid, _) => vid }

lpaGraph = org.apache.spark.graphx.impl.GraphImpl@604c94f0


org.apache.spark.graphx.impl.GraphImpl@604c94f0

In [4]:
//向邻居发送自己的标签号
def sendMessage(e: EdgeTriplet[VertexId, Int]): Iterator[(VertexId, Map[VertexId, Long])] = {
      Iterator((e.srcId, Map(e.dstAttr -> 1L)), (e.dstId, Map(e.srcAttr -> 1L)))
    }

sendMessage: (e: org.apache.spark.graphx.EdgeTriplet[org.apache.spark.graphx.VertexId,Int])Iterator[(org.apache.spark.graphx.VertexId, Map[org.apache.spark.graphx.VertexId,Long])]


In [5]:
//统计标签数
import scala.collection.{mutable, Map}
def mergeMessage(count1: Map[VertexId, Long], count2: Map[VertexId, Long])
      : Map[VertexId, Long] = {
      // Mimics the optimization of breakOut, not present in Scala 2.13, while working in 2.12
      val map = mutable.Map[VertexId, Long]()
      (count1.keySet ++ count2.keySet).foreach { i =>
        val count1Val = count1.getOrElse(i, 0L)
        val count2Val = count2.getOrElse(i, 0L)
        map.put(i, count1Val + count2Val)
      }
      map
    }

mergeMessage: (count1: scala.collection.Map[org.apache.spark.graphx.VertexId,Long], count2: scala.collection.Map[org.apache.spark.graphx.VertexId,Long])scala.collection.Map[org.apache.spark.graphx.VertexId,Long]


In [6]:
//将邻居中标签占比最高的标签更新为当前节点的标签
def vertexProgram(vid: VertexId, attr: Long, message: Map[VertexId, Long]): VertexId = {
      if (message.isEmpty) attr else message.maxBy(_._2)._1
    }

vertexProgram: (vid: org.apache.spark.graphx.VertexId, attr: Long, message: scala.collection.Map[org.apache.spark.graphx.VertexId,Long])org.apache.spark.graphx.VertexId


In [7]:
val initialMessage = Map[VertexId, Long]()
val maxSteps=10
// Pregel(lpaGraph, initialMessage, maxIterations = maxSteps)(
//       vprog = vertexProgram,
//       sendMsg = sendMessage,
//       mergeMsg = mergeMessage)
//这里如果是单机运行会有序列化问题，可以这样解决：1）集群环境不会有问题  2）将上面的仨函数直接隐式的写到pregel里面

initialMessage = Map()
maxSteps = 10


10

lpa缺点很多，主要俩点：   

1.需要自己设置最大迭代次数（上面的maxSteps），如果不设置，最后可能所有的顶点都是同一个标签；   
2.由于初始轮，所有邻居标签都只出现过一次，其实等价于从邻居标签中随机选择一个，这样会造成最终结果很不稳定，重复运行代码，每次的结果差异都会很大;